In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [29]:
db = pd.read_csv('dataset_join.csv')
db.head()

,id,fecha,comercio,giro_comercio,tipo_venta,monto,fecha_nacimiento,fecha_alta,id_municipio,id_estado,tipo_persona,genero,actividad_empresarial,tiempo_apertura,edad
0,91477f382c3cf63ab5cd9263b502109243741158,02/01/2022,AMAZON,COMERCIOS_ELECTRONICOS_(VTAS_POR_INTERNET),digital,5.99,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
1,91477f382c3cf63ab5cd9263b502109243741158,05/01/2022,RAPPI,SERVICIOS_EMPRESARIALES_-_NO_CLASIFICADOS,digital,13.01,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
2,91477f382c3cf63ab5cd9263b502109243741158,05/01/2022,RAPPI,SERVICIOS_EMPRESARIALES_-_NO_CLASIFICADOS,digital,15.84,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
3,91477f382c3cf63ab5cd9263b502109243741158,05/01/2022,AMAZON,COMERCIOS_ELECTRONICOS_(VTAS_POR_INTERNET),digital,8.17,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
4,91477f382c3cf63ab5cd9263b502109243741158,05/01/2022,AMAZON,COMERCIOS_ELECTRONICOS_(VTAS_POR_INTERNET),digital,2.54,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37


In [ ]:
db_sin_id = db.drop(['id'], axis=1)
db_sin_id.head()

,fecha,comercio,giro_comercio,tipo_venta,monto,fecha_nacimiento,fecha_alta,id_municipio,id_estado,tipo_persona,genero,actividad_empresarial,tiempo_apertura,edad
0,02/01/2022,AMAZON,COMERCIOS_ELECTRONICOS_(VTAS_POR_INTERNET),digital,5.99,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
1,05/01/2022,RAPPI,SERVICIOS_EMPRESARIALES_-_NO_CLASIFICADOS,digital,13.01,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
2,05/01/2022,RAPPI,SERVICIOS_EMPRESARIALES_-_NO_CLASIFICADOS,digital,15.84,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
3,05/01/2022,AMAZON,COMERCIOS_ELECTRONICOS_(VTAS_POR_INTERNET),digital,8.17,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
4,05/01/2022,AMAZON,COMERCIOS_ELECTRONICOS_(VTAS_POR_INTERNET),digital,2.54,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37


In [ ]:
#db_sin_id.hist(bins=50, figsize=(20,15))
#plt.show()

In [30]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337998 entries, 0 to 337997
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id                     337998 non-null  object 
 1   fecha                  337998 non-null  object 
 2   comercio               337998 non-null  object 
 3   giro_comercio          337998 non-null  object 
 4   tipo_venta             337998 non-null  object 
 5   monto                  337998 non-null  float64
 6   fecha_nacimiento       337998 non-null  object 
 7   fecha_alta             337998 non-null  object 
 8   id_municipio           337998 non-null  int64  
 9   id_estado              337998 non-null  int64  
 10  tipo_persona           337998 non-null  int64  
 11  genero                 337998 non-null  int64  
 12  actividad_empresarial  337998 non-null  object 
 13  tiempo_apertura        337998 non-null  int64  
 14  edad                   337998 non-nu

In [ ]:
db.loc[db['fecha'].apply(pd.to_datetime, errors='coerce').isna(), 'fecha'].unique()[:20]

<ipython-input-19-c347aaeac79a>:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  db.loc[db['fecha'].apply(pd.to_datetime, errors='coerce').isna(), 'fecha'].unique()[:20]


array([], dtype=object)

In [ ]:
# Lista hasta 20 valores de 'fecha' cuyo parseo falla,
# ignorando los que ya son NaN en origen.
mask_bad = (
    db['fecha'].notna() &                                # descarta NaN originales
    pd.to_datetime(db['fecha'], errors='coerce', dayfirst=True).isna()
)

print("Fechas con formato NO reconocido:", mask_bad.sum())
bad_examples = db.loc[mask_bad, 'fecha'].unique()[:20]
print(bad_examples)


Fechas con formato NO reconocido: 0
[]


In [31]:
# ─── 1. Cuantifica cuántas filas tienen problemas ──────────────────────────
total_filas = len(db)

# a) Fechas que ya venían vacías
sin_fecha  = db['fecha'].isna().sum()

# b) Montos que no se pudieron convertir a numérico
db['monto'] = pd.to_numeric(db['monto'], errors='coerce')
sin_monto = db['monto'].isna().sum()

print(f"Filas totales          : {total_filas:,}")
print(f"· sin fecha (NaT)      : {sin_fecha:,}")
print(f"· sin monto (NaN)      : {sin_monto:,}")
print(f"Filas que quedarían    : {total_filas - sin_fecha - sin_monto:,}")


Filas totales          : 337,998
· sin fecha (NaT)      : 0
· sin monto (NaN)      : 0
Filas que quedarían    : 337,998


In [32]:
from dateutil.parser import parse
from collections import Counter

# Solo fechas no nulas
fechas_unicas = db['fecha'].dropna().astype(str).unique()

formatos = []

for f in fechas_unicas[:5000]:  # evalúa solo los primeros 5000 para velocidad
    try:
        parsed = parse(f, dayfirst=False, fuzzy=False)
        formatos.append(parsed.strftime('%Y-%m-%d %H:%M:%S'))
    except Exception:
        formatos.append('ERROR')

# Cuenta los tipos de fecha detectados
conteo = Counter(formatos)
for fmt, count in conteo.most_common(10):
    print(f"{fmt} → {count} ocurrencias")


2022-02-01 00:00:00 → 1 ocurrencias
2022-05-01 00:00:00 → 1 ocurrencias
2022-06-01 00:00:00 → 1 ocurrencias
2022-07-01 00:00:00 → 1 ocurrencias
2022-11-01 00:00:00 → 1 ocurrencias
2022-01-13 00:00:00 → 1 ocurrencias
2022-01-14 00:00:00 → 1 ocurrencias
2022-01-15 00:00:00 → 1 ocurrencias
2022-01-18 00:00:00 → 1 ocurrencias
2022-01-21 00:00:00 → 1 ocurrencias


In [33]:
from collections import Counter
import re

def detectar_formato(fecha_str):
    """Extrae un patrón de formato de fecha simple desde el string original."""
    if re.match(r'^\d{4}-\d{2}-\d{2}', fecha_str):
        return 'YYYY-MM-DD'
    elif re.match(r'^\d{2}/\d{2}/\d{4}', fecha_str):
        return 'DD/MM/YYYY'
    elif re.match(r'^\d{4}/\d{2}/\d{2}', fecha_str):
        return 'YYYY/MM/DD'
    elif re.match(r'^\d{2}-\d{2}-\d{4}', fecha_str):
        return 'DD-MM-YYYY'
    elif re.match(r'^\d{4}-\d{2}-\d{2}T', fecha_str):
        return 'ISO: YYYY-MM-DDTHH:MM:SS'
    else:
        return 'OTRO'

# Tomamos solo las fechas no nulas y convertidas a string
fechas_raw = db['fecha'].dropna().astype(str)

# Detectamos el formato de cada una
formatos_detectados = fechas_raw.map(detectar_formato)

# Contamos cuántas hay de cada uno
conteo = formatos_detectados.value_counts()

print("Distribución de formatos detectados:")
print(conteo)


Distribución de formatos detectados:
fecha
DD/MM/YYYY    337998
Name: count, dtype: int64


In [34]:
db['fecha'] = pd.to_datetime(db['fecha'], errors='coerce', dayfirst=True)

In [35]:
print("Fechas convertidas a datetime (NaT):", db['fecha'].isna().sum())
print("Ejemplo convertida:", db['fecha'].iloc[0])


Fechas convertidas a datetime (NaT): 0
Ejemplo convertida: 2022-01-02 00:00:00


In [36]:
db.head(15)

,id,fecha,comercio,giro_comercio,tipo_venta,monto,fecha_nacimiento,fecha_alta,id_municipio,id_estado,tipo_persona,genero,actividad_empresarial,tiempo_apertura,edad
0,91477f382c3cf63ab5cd9263b502109243741158,2022-01-02,AMAZON,COMERCIOS_ELECTRONICOS_(VTAS_POR_INTERNET),digital,5.99,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
1,91477f382c3cf63ab5cd9263b502109243741158,2022-01-05,RAPPI,SERVICIOS_EMPRESARIALES_-_NO_CLASIFICADOS,digital,13.01,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
2,91477f382c3cf63ab5cd9263b502109243741158,2022-01-05,RAPPI,SERVICIOS_EMPRESARIALES_-_NO_CLASIFICADOS,digital,15.84,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
3,91477f382c3cf63ab5cd9263b502109243741158,2022-01-05,AMAZON,COMERCIOS_ELECTRONICOS_(VTAS_POR_INTERNET),digital,8.17,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
4,91477f382c3cf63ab5cd9263b502109243741158,2022-01-05,AMAZON,COMERCIOS_ELECTRONICOS_(VTAS_POR_INTERNET),digital,2.54,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
5,91477f382c3cf63ab5cd9263b502109243741158,2022-01-06,AMAZON,COMERCIOS_ELECTRONICOS_(VTAS_POR_INTERNET),digital,5.20,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
6,91477f382c3cf63ab5cd9263b502109243741158,2022-01-07,AMAZON,COMERCIOS_ELECTRONICOS_(VTAS_POR_INTERNET),digital,16.33,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
7,91477f382c3cf63ab5cd9263b502109243741158,2022-01-11,CFE,"SERV_GUBERNAM,_AGUA,_ELECTRICIDAD,PREDIAL,TENE...",digital,185.78,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
8,91477f382c3cf63ab5cd9263b502109243741158,2022-01-13,AMAZON,COMERCIOS_ELECTRONICOS_(VTAS_POR_INTERNET),digital,13.91,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
9,91477f382c3cf63ab5cd9263b502109243741158,2022-01-14,AMAZON,COMERCIOS_ELECTRONICOS_(VTAS_POR_INTERNET),digital,19.66,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37


In [39]:
db.to_csv('dataset_join_limpio.csv', index=False)

In [37]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337998 entries, 0 to 337997
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   id                     337998 non-null  object        
 1   fecha                  337998 non-null  datetime64[ns]
 2   comercio               337998 non-null  object        
 3   giro_comercio          337998 non-null  object        
 4   tipo_venta             337998 non-null  object        
 5   monto                  337998 non-null  float64       
 6   fecha_nacimiento       337998 non-null  object        
 7   fecha_alta             337998 non-null  object        
 8   id_municipio           337998 non-null  int64         
 9   id_estado              337998 non-null  int64         
 10  tipo_persona           337998 non-null  int64         
 11  genero                 337998 non-null  int64         
 12  actividad_empresarial  337998 non-null  obje

In [42]:
#comercio como entero
db['comercio'] = pd.factorize(db['comercio'])[0].astype('int32')
db.head(15)

,id,fecha,comercio,giro_comercio,tipo_venta,monto,fecha_nacimiento,fecha_alta,id_municipio,id_estado,tipo_persona,genero,actividad_empresarial,tiempo_apertura,edad
0,91477f382c3cf63ab5cd9263b502109243741158,2022-01-02,0,COMERCIOS_ELECTRONICOS_(VTAS_POR_INTERNET),digital,5.99,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
1,91477f382c3cf63ab5cd9263b502109243741158,2022-01-05,1,SERVICIOS_EMPRESARIALES_-_NO_CLASIFICADOS,digital,13.01,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
2,91477f382c3cf63ab5cd9263b502109243741158,2022-01-05,1,SERVICIOS_EMPRESARIALES_-_NO_CLASIFICADOS,digital,15.84,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
3,91477f382c3cf63ab5cd9263b502109243741158,2022-01-05,0,COMERCIOS_ELECTRONICOS_(VTAS_POR_INTERNET),digital,8.17,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
4,91477f382c3cf63ab5cd9263b502109243741158,2022-01-05,0,COMERCIOS_ELECTRONICOS_(VTAS_POR_INTERNET),digital,2.54,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
5,91477f382c3cf63ab5cd9263b502109243741158,2022-01-06,0,COMERCIOS_ELECTRONICOS_(VTAS_POR_INTERNET),digital,5.20,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
6,91477f382c3cf63ab5cd9263b502109243741158,2022-01-07,0,COMERCIOS_ELECTRONICOS_(VTAS_POR_INTERNET),digital,16.33,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
7,91477f382c3cf63ab5cd9263b502109243741158,2022-01-11,2,"SERV_GUBERNAM,_AGUA,_ELECTRICIDAD,PREDIAL,TENE...",digital,185.78,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
8,91477f382c3cf63ab5cd9263b502109243741158,2022-01-13,0,COMERCIOS_ELECTRONICOS_(VTAS_POR_INTERNET),digital,13.91,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
9,91477f382c3cf63ab5cd9263b502109243741158,2022-01-14,0,COMERCIOS_ELECTRONICOS_(VTAS_POR_INTERNET),digital,19.66,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37


In [43]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337998 entries, 0 to 337997
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   id                     337998 non-null  object        
 1   fecha                  337998 non-null  datetime64[ns]
 2   comercio               337998 non-null  int32         
 3   giro_comercio          337998 non-null  object        
 4   tipo_venta             337998 non-null  object        
 5   monto                  337998 non-null  float64       
 6   fecha_nacimiento       337998 non-null  object        
 7   fecha_alta             337998 non-null  object        
 8   id_municipio           337998 non-null  int64         
 9   id_estado              337998 non-null  int64         
 10  tipo_persona           337998 non-null  int64         
 11  genero                 337998 non-null  int64         
 12  actividad_empresarial  337998 non-null  obje

In [45]:
db.to_csv('new_base.csv', index=False)

In [50]:
# Obtener el arreglo original de categorías
comercios_unicos = pd.factorize(db['comercio'])[1]

# Crear un DataFrame de mapeo
mapa_comercios = pd.DataFrame({
    'comercio_le': range(len(comercios_unicos)),
    'comercio': comercios_unicos
})

# Mostrar los primeros
print(mapa_comercios.head(10))


   comercio_le  comercio
0            0         0
1            1         1
2            2         2
3            3         3
4            4         4
5            5         5
6            6         6
7            7         7
8            8         8
9            9         9


In [52]:
print(db['comercio'].unique()[:10])
print(db['comercio'].dtype)


[0 1 2 3 4 5 6 7 8 9]
int32


In [53]:
db.head()

,id,fecha,comercio,giro_comercio,tipo_venta,monto,fecha_nacimiento,fecha_alta,id_municipio,id_estado,tipo_persona,genero,actividad_empresarial,tiempo_apertura,edad
0,91477f382c3cf63ab5cd9263b502109243741158,2022-01-02,0,COMERCIOS_ELECTRONICOS_(VTAS_POR_INTERNET),digital,5.99,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
1,91477f382c3cf63ab5cd9263b502109243741158,2022-01-05,1,SERVICIOS_EMPRESARIALES_-_NO_CLASIFICADOS,digital,13.01,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
2,91477f382c3cf63ab5cd9263b502109243741158,2022-01-05,1,SERVICIOS_EMPRESARIALES_-_NO_CLASIFICADOS,digital,15.84,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
3,91477f382c3cf63ab5cd9263b502109243741158,2022-01-05,0,COMERCIOS_ELECTRONICOS_(VTAS_POR_INTERNET),digital,8.17,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37
4,91477f382c3cf63ab5cd9263b502109243741158,2022-01-05,0,COMERCIOS_ELECTRONICOS_(VTAS_POR_INTERNET),digital,2.54,21/09/1987,28/10/2019,6519019,19,0,1,PROFESIONISTA INDEPENDIENTE,5,37


In [54]:
df_original = pd.read_csv('dataset_join_limpio.csv')

df_original['comercio_le'], comercios_unicos = pd.factorize(df_original['comercio'])


In [55]:
mapa_comercios = pd.DataFrame({
    'comercio_le': range(len(comercios_unicos)),
    'comercio': comercios_unicos
})


In [56]:
mapa_comercios.to_csv('mapa_comercio_le.csv', index=False)